In [1]:
import pandas as pd
path ='/content/drive/MyDrive/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz'
df = pd.read_table(path, error_bad_lines=False)

b'Skipping line 5782: expected 15 fields, saw 22\nSkipping line 31979: expected 15 fields, saw 22\nSkipping line 32408: expected 15 fields, saw 22\nSkipping line 45709: expected 15 fields, saw 22\nSkipping line 64585: expected 15 fields, saw 22\n'
b'Skipping line 115742: expected 15 fields, saw 22\nSkipping line 125537: expected 15 fields, saw 22\nSkipping line 127580: expected 15 fields, saw 22\nSkipping line 128696: expected 15 fields, saw 22\n'
b'Skipping line 143851: expected 15 fields, saw 22\nSkipping line 147069: expected 15 fields, saw 22\nSkipping line 149233: expected 15 fields, saw 22\nSkipping line 167536: expected 15 fields, saw 22\nSkipping line 168535: expected 15 fields, saw 22\nSkipping line 171659: expected 15 fields, saw 22\nSkipping line 184533: expected 15 fields, saw 22\nSkipping line 184624: expected 15 fields, saw 22\nSkipping line 184908: expected 15 fields, saw 22\n'
b'Skipping line 219150: expected 15 fields, saw 22\nSkipping line 223178: expected 15 fields, 

In [2]:
n = len(df)

In [3]:
import collections
itemsPerUser = collections.defaultdict(list)
usersPerItem = collections.defaultdict(list)

for i in range(n):
  user,item = df['customer_id'][i],df['product_id'][i]
  itemsPerUser[user].append((item,i))
  usersPerItem[item].append((user,i))

In [4]:
average = sum([df['star_rating'][i] for i in range(n)])/n

In [5]:
labels = [df['star_rating'][i] for i in range(n)]

In [6]:
users = itemsPerUser.keys()
items = usersPerItem.keys()
nUsers = len(users)
nItems = len(items)

In [7]:
alpha = average
betaUser = collections.defaultdict(float)
betaItem = collections.defaultdict(float)

In [8]:
def MSE(predictions,label):
  error = sum([(x-y)**2 for x,y in zip(predictions,label)])
  return error/len(labels)

In [9]:
def predict(user,item):
  return alpha+betaUser[user]+betaItem[item]

In [10]:
def unpack(theta):
  global alpha
  global betaUser
  global betaItem
  alpha = theta[0]
  betaUser = dict(zip(users,theta[1:nUsers+1]))
  betaItem = dict(zip(items,theta[nUsers+ 1: ]))

In [11]:
def cost(theta,lamda,labels):
  unpack(theta)
  predicitons = [predict (df['customer_id'][i], df['product_id'][i]) for i in range(n) ]
  cost = MSE(predicitons,labels)
  print(cost)
  for i in users:
    cost+= lamda*(betaUser[i]**2)
  for j in items:
    cost+= lamda*(betaItem[j]**2)
  return cost

In [12]:
import numpy as np
def derivate(theta,lamda,labels):
  unpack(theta)
  daplha = 0
  dBetaUser = collections.defaultdict(float)
  dBetaItem = collections.defaultdict(float)
  for i in range(n):
    user,item = df['customer_id'][i],df['product_id'][i]
    prediction = predict(user,item)
    diff = prediction-df['star_rating'][i]
    daplha += 2/n*diff
    dBetaUser[user]+=2/n*diff
    dBetaItem[item]+=2/n*diff
  for us in users:
    dBetaUser[us]+=2*lamda*betaUser[us]
  for it in items:
    dBetaItem[it]+=2*lamda*betaItem[it]
  dtheta = [daplha]+[dBetaUser[i] for i in users] + [dBetaItem[i] for i in items]
  return np.array(dtheta) 

In [14]:
from scipy import optimize
optimize.fmin_l_bfgs_b(func = cost, x0 = [average]+[0.0]*(nUsers+nItems),fprime = derivate,args= (0.01,labels))

1.4796817958151918
1.4687530422718669
1.50136599773868
1.469178608433825
1.473510231463674
1.4734745246098198
1.4734041525744686


(array([ 4.24912003e+00, -1.38068220e-04,  5.79785117e-04, ...,
         8.29433475e-05, -2.20507029e-04, -2.75262418e-05]),
 1.4764214008348497,
 {'funcalls': 7,
  'grad': array([-1.85488594e-06,  2.71922809e-09, -1.07298707e-08, ...,
         -1.64628186e-09,  4.08671817e-09,  5.01438362e-10]),
  'nit': 5,
  'task': b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'warnflag': 0})